In [1]:
from astropy.io import fits 
import matplotlib.pyplot as plt
import numpy as np 
from scipy.optimize import minimize, rosen, rosen_der
from scipy import interpolate
import scipy
import os 
from scipy.optimize import curve_fit
from scipy.integrate import simps
import math
from astropy.cosmology import WMAP9 as cosmo
from astroquery.mast import Observations
import csv

In [ ]:
#https://mast.stsci.edu/api/v0/_c_a_o_mfields.html
#https://astroquery.readthedocs.io/en/latest/mast/mast.html#id1
#https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html
#https://outerspace.stsci.edu/display/MASTDOCS/Search+a+List+of+Targets
#https://www.stsci.edu/hst/instrumentation/wfc3/performance/throughputs

In [2]:
#madcows1 = fits.open("C:/Users/19133/Documents/cows_official.fits")
madcows = fits.open("C:/Users/19133/Documents/MC2.fits")

In [3]:
with open('C:/Users/19133/Documents/MAST_ACS_07_17.csv', 'w', newline='') as csvfile:
    fieldnames = ['obsid','MADCOW_id','file']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

In [4]:
len(madcows[1].data['RA'])

24646

In [ ]:
failed = [] 
for x in range(0,24647):
    try: 
        obs_table = Observations.query_object(str(madcows[1].data['RA'][x]) + " " + str(madcows[1].data['DEC'][x]),radius="0.03 deg")
        inds = np.where(obs_table['instrument_name'] == "ACS/WFC")[0]
        if len(inds) > 0:
            for y in obs_table[inds]['obsid']:
                data_products = Observations.get_product_list(y)
                products = Observations.filter_products(data_products,productType=["PREVIEW"])
    
                manifest = Observations.download_products(products[0])
                    
                manifests = manifest[0][0]
                filename = manifests.split("\\")[-1]
                
                with open('C:/Users/19133/Documents/MAST_ACS_07_17.csv', 'a', newline='') as csvfile:
                    fieldnames = ['obsid','MADCOW_id','file']
                    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                    writer.writerow({'obsid':y,'MADCOW_id':x,'file':filename})
        else:
            True 
    except: 
        failed.append(x)

In [ ]:
import glob
for name in glob.glob('C:/Users/19133/Documents/Research/mastDownload/*/*/*'):
    basename = os.path.basename(name)
    os.rename(name, "C:/Users/19133/Documents/Research/mastDownload/"+basename)

In [ ]:
from astropy.io import ascii
data = ascii.read('C:/Users/19133/Documents/MAST_matching_targets_07_08.csv')

In [ ]:
pretties = np.array(['hlsp_clash_hst_wfc3ir_macs1115_f105w_v1_drz.jpg','hlsp_clash_hst_wfc3ir-30mas_a209_f105w_v1_drz.jpg','hst_11696_da_wfc3_ir_f140w_ib8cda_drz.jpg','hst_12051_01_wfc3_ir_f125w_01_drz.jpg','hst_12064_0p_wfc3_ir_f125w_ibew0p_drz.jpg','hst_12166_8q_wfc3_ir_f110w_ibkz8q_drz.jpg','hst_12453_a4_wfc3_ir_f160w_drz.jpg','hst_12488_1v_wfc3_ir_f110w_drz.jpg','hst_12488_51_wfc3_ir_f110w_ibs451_drz.jpg','hst_12578_09_wfc3_ir_f160w_ibra09_drz.jpg','hst_13003_01_wfc3_ir_f125w_ic2i01_drz.jpg','hst_13003_20_wfc3_ir_f125w_drz.jpg','hst_13023_47_wfc3_ir_f140w_drz.jpg','hst_13046_j1_wfc3_ir_f125w_drz.jpg','hst_13177_04_wfc3_ir_f125w_ic6i04_drz.jpg','hst_13177_05_wfc3_ir_f098m_drz.jpg','hst_13671_1h_wfc3_ir_f110w_icoz1h_drz.jpg','hst_13677_49_wfc3_ir_f140w_drz.jpg','hst_14096_gb_wfc3_ir_f105w_iczggb_drz.jpg','hst_14096_jl_wfc3_ir_f105w_iczgjl_drz.jpg','hst_14096_n2_wfc3_ir_f125w_iczgn2_drz.jpg','hst_14098_1s_wfc3_ir_f110w_drz.jpg','hst_15132_1k_wfc3_ir_f110w_idkl1k_drz.jpg','hst_15242_07_wfc3_ir_f110w_idmh07jk_drz.jpg','hst_15253_08_wfc3_ir_f160w_idki08_drz.jpg','hst_15842_1a_wfc3_ir_f110w_ie4y1aen_drz.jpg','hst_15867_10_wfc3_ir_f140w_ie5010kr_drz.jpg','hst_15867_57_wfc3_ir_f140w_ie5057fc_drz.jpg','hst_15867_77_wfc3_ir_f140w_ie5077_drz.jpg','hst_15867_79_wfc3_ir_f140w_ie5079_drz.jpg','hst_15867_82_wfc3_ir_f140w_ie5082_drz.jpg','hst_15867_85_wfc3_ir_f140w_ie5085_drz.jpg','hst_16668_11_wfc3_ir_f110w_ieov11g7_drz.jpg','hst_16729_0d_wfc3_ir_f110w_iepl0dnf_drz.jpg','hst_16729_0e_wfc3_ir_f110w_iepl0e_drz.jpg','hst_16773_34_wfc3_ir_f140w_ienh34_drz.jpg','hst_16773_b4_wfc3_ir_f140w_ienhb4_drz.jpg',
                     'hst_16773_a7_wfc3_ir_f140w_ienha7i8_drz.jpg'])
for x in range(len(data['file'])):
    if data['file'][x][0] == 'i':
        pretties = np.append(pretties,data['file'][x])

In [ ]:
pretty_obsids = []
for x in pretties:
    ind = np.where(data['file'] == x)[0]
    for y in ind:
        pretty_obsids.append(data['obsid'][y])

In [ ]:
for y in pretty_obsids:
    Observations.download_products(str(y),productType=["PREVIEW"])

In [ ]:
#used this in online interface
import csv

with open('C:/Users/19133/Documents/madcows_targets_6_29.csv', 'w', newline='') as csvfile:
    fieldnames = ['RA', 'DEC']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for x in range(len(madcows[1].data['RA'])):
        writer.writerow({'RA': madcows[1].data['RA'][x], 'DEC': madcows[1].data['DEC'][x]})

In [ ]:
from astropy.table import Table
matches = Table.read("MAST_matching_targets.csv")
obsids = matches['obsid']

for x in [0,1,2]:
    data_products = Observations.get_product_list(str(obsids[x]))
    #print(data_products['dataURI'])
    manifest = Observations.download_products(data_products, productType="SCIENCE")